___
___
___

<span style='color:blue'> <font size="8"> 
**PyTorch at Scale with Azure ML** 
</font> </span>
___

This notebook provides an introduction to run PyTorch training scripts at enterprise scale using Azure ML.
It consists of the following sections:

#### Experiment Setup
> * Import packages
> * Initialize a workspace
> * Get the data
> * Prepare training script
> * Create a compute target
> * Define your environment
#### Run Training  
> * Create a ScriptRunConfig
> * Submit your run
> * Register or download a model
#### Distributed Training 
> * Using Horovod
> * Using DistributedDataParallel
> * Troubleshooting

The example scripts are used to classify chicken and turkey images to build a deep learning neural network (DNN) based on [PyTorch transfer learning](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) tutorial. Transfer learning is a technique that applies knowledge gained from solving one problem to a different but related problem. This shortcuts the training process by requiring less data, time, and compute resources than training from scratch. See the [deep learning vs machine learning](https://docs.microsoft.com/en-us/azure/machine-learning/concept-deep-learning-vs-machine-learning) article to learn more about transfer learning.

Whether you're training a deep learning PyTorch model from the ground-up or you're bringing an existing model into the cloud, you can use Azure Machine Learning to scale out open-source training jobs using elastic cloud compute resources. You can build, deploy, version, and monitor production-grade models with Azure ML.

**Note 1:** This notebook is a living script that will be modified and extended continuously. 

**Note 2:** While you can run this notebook in VS, you will get the best outcome by running it in Jupyter.

<span style='color:Red'> **Disclaimer:** The materials in this notebook are replicated from different online sources, and thus intends for individual upskilling only, not for corporate or commercial distribution. </span>

___
# <span style='color:Blue'> Experiment Setup </span>
This section sets up the training experiment by loading the required Python packages, initializing a workspace, creating the compute target, and defining the training environment.

### Import packages
First, import the necessary Python libraries:

In [1]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

### Initialize a workspace
The Azure Machine Learning workspace is the top-level resource for the service. It provides us with a centralized place to work with all the artifacts we create. In the Python SDK, you can access the workspace artifacts by creating a workspace object.

Create a workspace object from the `config.json` file that you can download from your AzureML portal:

In [2]:
ws = Workspace.from_config()

### Get the data
The dataset consists of about 120 training images each for turkeys and chickens, with 100 validation images for each class. We will download and extract the dataset as part of our training script `pytorch_train.py`. The images are a subset of the [Open Images v5 Dataset](https://storage.googleapis.com/openimages/web/index.html).

### Prepare training script
Here we already provide the training script `pytorch_train.py`, but in practice, we can take any custom training script, as is, and run it with Azure ML. Let's create a folder for our training script(s):

In [3]:
project_folder = './pytorch-birds'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('pytorch_train.py', project_folder)

'./pytorch-birds\\pytorch_train.py'

### Create a compute target
Let's create a compute target for our PyTorch job to run on. In this example, we create a GPU-enabled Azure ML compute cluster:

In [4]:
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target


Note that you may choose to use low-priority VMs to run some or all of your workloads to [lower your compute cluster cost](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python#low-pri-vm). You may find more information on compute targets in the [what is a compute target](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target) article.

### Define your environment
To define the Azure ML Environment that encapsulates your training script's dependencies, you can either define a custom environment or use an Azure ML curated environment.

#### Option 1: Use a curated environment
Azure ML provides prebuilt, [curated environments](https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments) if you don't want to define your own environment. Curated environments are provided by Azure ML and are available in your workspace by default. They are backed by cached Docker images that use the latest version of the Azure ML SDK, reducing the run preparation cost and allowing for faster deployment time. Azure ML has several CPU and GPU curated environments for PyTorch corresponding to different versions of PyTorch. 

To use a curated environment, we can run the following command:

In [5]:
curated_env_name = 'AzureML-PyTorch-1.6-GPU'
pytorch_env = Environment.get(workspace=ws, name=curated_env_name)

To see the packages included in the curated environment, we can write out the conda dependencies to disk:

In [6]:
pytorch_env.save_to_directory(path=curated_env_name, overwrite=True)

Make sure the curated environment includes all the dependencies required by your training script. If not, you will have to modify the environment to include the missing dependencies. Note that if the environment is modified, you will have to give it a new name, as the 'AzureML' prefix is reserved for curated environments. If you modified the conda dependencies YAML file, you can create a new environment from it with a new name, e.g.:

If you had instead modified the curated environment object directly, you can clone that environment with a new name:

#### Option 2: Create a custom environment
We can also create our own Azure ML environment that encapsulates our training script's dependencies.

First, we need to define our conda dependencies in a YAML file, named `conda_dependencies.yml` in our case:

        channels:
        - conda-forge
        dependencies:
        - python=3.6.2
        - pip:
          - azureml-defaults
          - torch==1.6.0
          - torchvision==0.7.0
          - future==0.17.1
          - pillow
      
Then we create an Azure ML environment from this conda environment specification. The environment will be packaged into a Docker container at runtime.

By default if no base image is specified, Azure ML will use a CPU image `azureml.core.environment.DEFAULT_CPU_IMAGE` as the base image. Since this example runs training on a GPU cluster, we will need to specify a GPU base image that has the necessary GPU drivers and dependencies. Azure ML maintains a set of base images published on Microsoft Container Registry (MCR) that we can use (see the [Azure/AzureML-Containers](https://github.com/Azure/AzureML-Containers) GitHub repo for more information).

Optionally, you can just capture all your dependencies directly in a custom Docker image or Dockerfile, and create your environment from that. For more information, see the [train with custom image](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-with-custom-image) article.

For more information on creating and using environments, see the [how to use environments](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments) article.

___
# <span style='color:Blue'> Run Training </span>

### Create a ScriptRunConfig
Let's create a `ScriptRunConfig` object to specify the configuration details of our training job, including our training script, environment to use, and the compute target to run on. Any arguments to our training script will be passed via command line if specified in the arguments parameter:

In [7]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='pytorch_train.py',
                      arguments=['--num_epochs', 30, '--output_dir', './outputs'],
                      compute_target=compute_target,
                      environment=pytorch_env)

For more information on configuring jobs with ScriptRunConfig, see the [how to setup training targets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets) article.

Note that Azure ML runs training scripts by copying the entire source directory. If you have sensitive data that you don't want to upload, use a [.ignore file](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-save-write-experiment-files#storage-limits-of-experiment-snapshots) or don't include it in the source directory . Instead, you can access your data using an Azure ML [dataset](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-with-datasets).

Also note that if you were previously using the PyTorch estimator to configure your PyTorch training jobs, please note that Estimators have been deprecated as of the 1.19.0 SDK release. With Azure ML SDK >= 1.15.0, ScriptRunConfig is the recommended way to configure training jobs, including those using deep learning frameworks. For common migration questions, see the [how to migrate from estimators to ScriptRunConfig](how-to-migrate-from-estimators-to-scriptrunconfig).

### Submit your run
The [run object](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py) provides the interface to the run history while the job is running and after it has completed:

In [8]:
run = Experiment(ws, name='AzureML-pytorch-birds').submit(src)
run.wait_for_completion(show_output=True)

RunId: AzureML-pytorch-birds_1618367837_5c70b7c9
Web View: https://ml.azure.com/experiments/AzureML-pytorch-birds/runs/AzureML-pytorch-birds_1618367837_5c70b7c9?wsid=/subscriptions/7302eae5-0528-426b-b207-3c5686fb8237/resourcegroups/SNCF/workspaces/dubaimetroml

Streaming azureml-logs/55_azureml-execution-tvmps_2cbca433c56a4256d60d3828c930cdb8649f93bf50a5cdc1025c811278c093b2_d.txt

2021-04-14T02:40:16Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root/jobs/dubaimetroml/azureml/azureml-pytorch-birds_1618367837_5c70b7c9/mounts/workspaceblobstore
2021-04-14T02:40:17Z Failed to start nvidia-fabricmanager due to exit status 5 with output Failed to start nvidia-fabricmanager.service: Unit nvidia-fabricmanager.service not found.
. Please ignore this if the GPUs don't utilize NVIDIA® NVLink® switches.
2021-04-14T02:40:17Z Starting output-watcher...
2021-04-14T02:40:17Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-04-14T02:40:18Z Executing '

48.9%
49.0%
49.0%
49.0%
49.0%
49.0%
49.1%
49.1%
49.1%
49.1%
49.1%
49.1%
49.2%
49.2%
49.2%
49.2%
49.2%
49.2%
49.3%
49.3%
49.3%
49.3%
49.3%
49.4%
49.4%
49.4%
49.4%
49.4%
49.4%
49.5%
49.5%
49.5%
49.5%
49.5%
49.5%
49.6%
49.6%
49.6%
49.6%
49.6%
49.6%
49.7%
49.7%
49.7%
49.7%
49.7%
49.8%
49.8%
49.8%
49.8%
49.8%
49.8%
49.9%
49.9%
49.9%
49.9%
49.9%
49.9%
50.0%
50.0%
50.0%
50.0%
50.0%
50.1%
50.1%
50.1%
50.1%
50.1%
50.1%
50.2%
50.2%
50.2%
50.2%
50.2%
50.2%
50.3%
50.3%
50.3%
50.3%
50.3%
50.3%
50.4%
50.4%
50.4%
50.4%
50.4%
50.5%
50.5%
50.5%
50.5%
50.5%
50.5%
50.6%
50.6%
50.6%
50.6%
50.6%
50.6%
50.7%
50.7%
50.7%
50.7%
50.7%
50.8%
50.8%
50.8%
50.8%
50.8%
50.8%
50.9%
50.9%
50.9%
50.9%
50.9%
50.9%
51.0%
51.0%
51.0%
51.0%
51.0%
51.0%
51.1%
51.1%
51.1%
51.1%
51.1%
51.2%
51.2%
51.2%
51.2%
51.2%
51.2%
51.3%
51.3%
51.3%
51.3%
51.3%
51.3%
51.4%
51.4%
51.4%
51.4%
51.4%
51.4%
51.5%
51.5%
51.5%
51.5%
51.5%
51.6%
51.6%
51.6%
51.6%
51.6%
51.6%
51.7%
51.7%
51.7%
51.7%
51.7%
51.7%
51.8%
51.8%
51.8%
51.8%
51.8%
51.9

train Loss: 0.5304 Acc: 0.7739
val Loss: 0.3380 Acc: 0.8640

Epoch 3/29
----------
train Loss: 0.5485 Acc: 0.7913
val Loss: 0.7015 Acc: 0.7800

Epoch 4/29
----------
train Loss: 0.5598 Acc: 0.7739
val Loss: 0.4609 Acc: 0.8440

Epoch 5/29
----------
train Loss: 0.7132 Acc: 0.7870
val Loss: 0.8479 Acc: 0.8200

Epoch 6/29
----------
train Loss: 0.5791 Acc: 0.8217
val Loss: 0.3654 Acc: 0.9040

Epoch 7/29
----------
train Loss: 0.3582 Acc: 0.8652
val Loss: 0.3357 Acc: 0.9200

Epoch 8/29
----------
train Loss: 0.2886 Acc: 0.9000
val Loss: 0.3497 Acc: 0.9080

Epoch 9/29
----------
train Loss: 0.3278 Acc: 0.8609
val Loss: 0.3040 Acc: 0.9160

Epoch 10/29
----------
train Loss: 0.3164 Acc: 0.9000
val Loss: 0.3145 Acc: 0.9080

Epoch 11/29
----------
train Loss: 0.3407 Acc: 0.8652
val Loss: 0.3016 Acc: 0.9080

Epoch 12/29
----------
train Loss: 0.2474 Acc: 0.9000
val Loss: 0.3070 Acc: 0.9040

Epoch 13/29
----------
train Loss: 0.3503 Acc: 0.8783
val Loss: 0.2959 Acc: 0.9080

Epoch 14/29
----------

{'runId': 'AzureML-pytorch-birds_1618367837_5c70b7c9',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-14T02:40:18.558441Z',
 'endTimeUtc': '2021-04-14T02:45:02.495461Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '29c7970a-b9ab-4a9b-bc7a-dba9bae8a738',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'pytorch_train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--num_epochs', '30', '--output_dir', './outputs'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'gpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'AzureML-Py

As the run is executed, it goes through the following stages:

* **Preparing:** A docker image is created according to the environment defined. The image is uploaded to the workspace's container registry and cached for later runs. Logs are also streamed to the run history and can be viewed to monitor progress. If a curated environment is specified instead, the cached image backing that curated environment will be used.

* **Scaling:** The cluster attempts to scale up if the Batch AI cluster requires more nodes to execute the run than are currently available.

* **Running:** All scripts in the script folder are uploaded to the compute target, data stores are mounted or copied, and the script is executed. Outputs from stdout and the `./logs` folder are streamed to the run history and can be used to monitor the run.

* **Post-Processing:** The `./outputs` folder of the run is copied over to the run history.

### Register or download a model
Once we've trained the model, we can register it to your workspace. Model registration lets you store and version our models in our workspace to simplify model management and deployment:

In [9]:
model = run.register_model(model_name='pytorch-birds', model_path='outputs/model.pt')

Note that the deployment how-to contains a section on registering models, but we can skip directly to [creating a compute target](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli#choose-a-compute-target) for deployment, since we already have a registered model.

We can also download a local copy of the model by using the Run object. In the training script `pytorch_train.py`, a PyTorch save object persists the model to a local folder (local to the compute target), where we can use the Run object to download a copy:

In [10]:
# Create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

# Download the model from run history
run.download_file(name='outputs/model.pt', output_file_path='./model/model.pt'),

(None,)

___
#  <span style='color:Blue'> Distributed Training </span>
Azure ML also supports multi-node distributed PyTorch jobs so that we can scale our training workloads. We can easily run distributed PyTorch jobs and let Azure ML manage the orchestration for us.

Azure ML supports running distributed PyTorch jobs with both Horovod and PyTorch's built-in DistributedDataParallel module.

### Using Horovod
Horovod is an open-source, all reduce framework for distributed training developed by Uber. It offers an easy path to writing distributed PyTorch code for training. For more information, refer to the [Horovod with PyTorch](https://horovod.readthedocs.io/en/stable/pytorch.html) documentation.

Our training code will have to be instrumented with Horovod for distributed training. Additionally, we need to make sure our training environment includes the horovod package. If you are using a PyTorch curated environment, horovod is already included as one of the dependencies. If you are using your own environment, make sure the horovod dependency is included, for example:

        channels:
        - conda-forge
        dependencies:
        - python=3.6.2
        - pip:
          - azureml-defaults
          - torch==1.6.0
          - torchvision==0.7.0
          - horovod==0.19.5

To execute a distributed job using MPI/Horovod on Azure ML, we must specify an [MpiConfiguration](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfig.mpiconfiguration?view=azure-ml-py) to the `distributed_job_config` parameter of the ScriptRunConfig constructor. If we want to run multiple processes per node (i.e. if our cluster SKU has multiple GPUs), we must also specify the `process_count_per_node` parameter in MpiConfiguration (the default is $1$). For a full tutorial on running distributed PyTorch with Horovod on Azure ML, see the [distributed PyTorch with Horovod](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/ml-frameworks/pytorch/distributed-pytorch-with-horovod) notebook.

Let's configure a 2-node distributed job running one process per node: 

In [11]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import MpiConfiguration

src = ScriptRunConfig(source_directory=project_folder,
                      script='pytorch_horovod_mnist.py',
                      compute_target=compute_target,
                      environment=pytorch_env,
                      distributed_job_config=MpiConfiguration(node_count=2))

### Using DistributedDataParallel
If we are using PyTorch's built-in [DistributedDataParallel](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html) module that is built using the `torch.distributed` package in our training code, we can also launch the distributed job via Azure ML. To launch a distributed PyTorch job on Azure ML, we have two options:

* **Per-process launch:** specify the total number of worker processes we want to run, and Azure ML will handle launching each process.

* **Per-node launch:** provide the `torch.distributed.launch` command that we want to run on each node, and the torch launch utility will handle launching the worker processes on each node.

There are no fundamental differences between these launch options; it is largely up to the user's preference or the conventions of the frameworks/libraries built on top of vanilla PyTorch (such as Lightning or Hugging Face).

#### Option 1: Per-process launch
To use this option to run a distributed PyTorch job, we do the following:

1. Specify the training script and arguments
1. Create a PyTorchConfiguration and specify the `process_count` as well as `node_count`. The `process_count` corresponds to the total number of processes you want to run for your job. This should typically equal the number of GPUs per node multiplied by the number of nodes. If `process_count` is not specified, Azure ML will by default launch one process per node.

Azure ML will set the following environment variables:

* `MASTER_ADDR` - IP address of the machine that will host the process with rank 0.
* `MASTER_PORT` - A free port on the machine that will host the process with rank 0.
* `NODE_RANK` - The rank of the node for multi-node training. The possible values are 0 to (total # of nodes - 1).
* `WORLD_SIZE` - The total number of processes, which should be equal to the total number of devices (GPU) used for distributed training.
* `RANK` - The (global) rank of the current process. The possible values are 0 to (world size - 1).
* `LOCAL_RANK` - The local (relative) rank of the process within the node. The possible values are 0 to (# of processes on the node - 1).

Since the required environment variables will be set for us by Azure ML, we can use the [default environment variable initialization](https://pytorch.org/docs/stable/distributed.html#environment-variable-initialization) method to initialize the process group in our training code.

The following code snippet configures a 2-node, 2-process-per-node PyTorch job:

In [12]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration

curated_env_name = 'AzureML-PyTorch-1.6-GPU'
pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
distr_config = PyTorchConfiguration(process_count=4, node_count=2)

src = ScriptRunConfig(
  source_directory='./src',
  script='pytorch_train.py',
  arguments=['--epochs', 25],
  compute_target=compute_target,
  environment=pytorch_env,
  distributed_job_config=distr_config,
)

run = Experiment(ws, 'experiment_name').submit(src)

To use this option for multi-process-per-node training, we will need to use Azure ML Python SDK >= 1.22.0, as `process_count` was introduced in 1.22.0.

If our training script passes information like local rank or rank as script arguments, we can reference the environment variable(s) in the arguments: `arguments=['--epochs', 50, '--local_rank', $LOCAL_RANK]`.

#### Option 2: Per-node launch
PyTorch provides a launch utility in `torch.distributed.launch` that users can use to launch multiple processes per node. The `torch.distributed.launch` module will spawn multiple training processes on each of the nodes. 

The following steps will demonstrate how to configure a PyTorch job with a per-node-launcher on Azure ML that will achieve the equivalent of running the following command in the shell:

        python -m torch.distributed.launch --nproc_per_node <num processes per node> \
          --nnodes <num nodes> --node_rank $NODE_RANK --master_addr $MASTER_ADDR \
          --master_port $MASTER_PORT --use_env \
          <your training script> <your script arguments>
      
1. Provide the `torch.distributed.launch` command to the command parameter of the `ScriptRunConfig` constructor. Azure ML will run this command on each node of your training cluster, where `--nproc_per_node` should be less than or equal to the number of GPUs available on each node. `MASTER_ADDR`, `MASTER_PORT`, and `NODE_RANK` are all set by Azure ML, so we can just reference the environment variables in the command. Azure ML sets `MASTER_PORT` to 6105, but you can pass a different value to the `--master_port` argument of `torch.distributed.launch` command if you wish. (The launch utility will reset the environment variables.)
1. Create a `PyTorchConfiguration` and specify the `node_count`. You do not need to set `process_count` as Azure ML will default to launching one process per node, which will run the launch command you specified.

In [13]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration

curated_env_name = 'AzureML-PyTorch-1.6-GPU'
pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
distr_config = PyTorchConfiguration(node_count=2)
launch_cmd = "python -m torch.distributed.launch --nproc_per_node 2 --nnodes 2 --node_rank $NODE_RANK --master_addr $MASTER_ADDR --master_port $MASTER_PORT --use_env train.py --epochs 50".split()

src = ScriptRunConfig(
  source_directory='./src',
  command=launch_cmd,
  compute_target=compute_target,
  environment=pytorch_env,
  distributed_job_config=distr_config,
)

run = Experiment(ws, 'experiment_name').submit(src)

For a full tutorial on running distributed PyTorch on Azure ML, see [Distributed PyTorch with DistributedDataParallel](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/ml-frameworks/pytorch/distributed-pytorch-with-distributeddataparallel).

### Troubleshooting
* **Horovod has been shut down:** In most cases, if you encounter `AbortedError: Horovod has been shut down`, there was an underlying exception in one of the processes that caused Horovod to shut down. Each rank in the MPI job gets it own dedicated log file in Azure ML. These logs are named `70_driver_logs`. In case of distributed training, the log names are suffixed with `_rank` to make it easier to differentiate the logs. To find the exact error that caused Horovod to shut down, go through all the log files and look for `Traceback` at the end of the driver_log files. One of these files will give you the actual underlying exception.

##  Export to ONNX 
To optimize inference with the [ONNX Runtime](https://docs.microsoft.com/en-us/azure/machine-learning/concept-onnx), convert your trained PyTorch model to the ONNX format. Inference, or model scoring, is the phase where the deployed model is used for prediction, most commonly on production data. See the [tutorial](https://github.com/onnx/tutorials/blob/master/tutorials/PytorchOnnxExport.ipynb) for an example.